# 라이브러리 설치

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# 데이터 준비

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터셋 불러오기
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/데캡디/review_link10.csv')

# 데이터 전처리

In [ ]:
# 해당 문장 제거
string =  '※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.'

review_clean = []
for i in range(len(df)):
    temp = df['리뷰'].iloc[i]
    review_clean.append(temp.replace(string, ""))

df['리뷰'] = review_clean

df['리뷰'] = df['리뷰'].str.replace('\n',' ')
df['리뷰'] = df['리뷰'].str.replace('  ',' ')

target_string = ['피부타입', '복합성에 좋아요', '피부고민', '진정에 좋아요', '자극도', '자극없이 순해요']
df = df[~df['리뷰'].map(lambda x: all(string in x for string in target_string))]

In [ ]:
import nltk
import konlpy
from konlpy.utils import pprint
from konlpy.tag import Mecab

from tqdm import tqdm, tqdm_notebook

In [ ]:
'''
import re
# 기호, 숫자, 영어 등은 다 제외하고 한글만 남김
df['리뷰'] = [re.sub('[0-9]+', '', str(e)) for e in df['리뷰']]
df['리뷰'] = df['리뷰'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

df.head()


# 형태소 분석 # 명사 추출

def clean_text(text):
    text = text.replace(".", " ").strip()
    text = text.replace(".", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+'
    text = re.sub(pattern=pattern, repl="", string=text)
    return text

def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1]
    return nouns

def tokenize(df):
    tokenizer = Mecab()
    processed_data = []
    for sent in tqdm(df['리뷰']):
        sentence = clean_text(sent.replace('\n', "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data
'''

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")
print(kospacing_sent)

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

In [ ]:
spelled_sent = spell_checker.check("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")

hanspell_sent = spelled_sent.checked
print(hanspell_sent)
print(kospacing_sent) # 앞서 사용한 kospacing 패키지에서 얻은 결과

In [ ]:
df_product_name = df['상품명'].unique()
df = df[df['상품명'] == df_product_name[0]]
df

# 형태소 분석

In [ ]:
# 형태소 분석 # 명사 추출
import re
from pykospacing import Spacing
from hanspell import spell_checker
spacing = Spacing()
def clean_text(text):
    text = text.replace(".", " ").strip()
    text = text.replace(".", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+'
    text = re.sub(pattern=pattern, repl="", string=text)
    return text

def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1]
    return nouns

def tokenize(df):
    tokenizer = Mecab()
    processed_data = []
    for sent in tqdm(df['리뷰']):
        sentence = clean_text(sent.replace('\n', "").strip())
        #kospacing_sent = spacing(sentence)
        spelled_sent = spell_checker.check(sentence)
        hanspell_sent = spelled_sent.checked
        processed_data.append(get_nouns(tokenizer, hanspell_sent))
    return processed_data

In [ ]:
processed_data = tokenize(df)

In [ ]:
import pickle

with open('/content/drive/MyDrive/데캡디/data_0.pickle', 'wb') as f:
    pickle.dump(processed_data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/MyDrive/데캡디/data_0.pickle', 'rb') as f:
    data = pickle.load(f)

In [ ]:
df['리뷰'].iloc[2]

In [ ]:
processed_data[2]

In [ ]:
vocab = []
for i in data:
    for j in i:
        vocab.append(j) 

In [ ]:
vocab

In [ ]:
my_set = set(vocab)
my_list = list(my_set)
my_list

In [ ]:
len(processed_data)

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()


In [ ]:
'''
noun_words = []
verb_words = []
for i in range(len(temp)):
  for word,pos in mecab.pos(temp[i]):
    if pos[:1]=='N':
      noun_words.append(word)
    elif pos[:1]=='V':
      verb_words.append(word)
'''

In [ ]:
'''
verb_words = {}
for i in range(len(temp)):
  for word,pos in mecab.pos(temp[i]):
    if pos[:1]=='V':
      verb_words[pos] = []
for i in range(len(temp)):
  for word,pos in mecab.pos(temp[i]):
    if pos[:1]=='V':
      verb_words[pos].append(word)
'''

In [ ]:
for pos in verb_words.keys():
  verb_words[pos] = list(set(verb_words[pos]))

In [ ]:
result = []
for i in range(len(df)):
  result.append(mecab.nouns(df.iloc[i,0]))

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
processed_data

In [ ]:
stop_words = "올리브 전체 사용 생각 제품 추가 정도"

In [ ]:
common_texts_and_tags = [
    (text, [f"str_{i}",]) for i, text in enumerate(data)
]
print("##"*20)
print("tags and its texts")
print("##"*20)
for text, tags in common_texts_and_tags:
    print(f"tags: {tags}, text: {text}")

In [ ]:
# words = 단어 list, tags = 문서ID
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in common_texts_and_tags]

In [ ]:
model = Doc2Vec(TRAIN_documents, vector_size=100, window=5, epochs=40, min_count=5, workers=4)

In [ ]:
for text, tags in common_texts_and_tags:
    trained_doc_vec = model.docvecs[tags[0]]
    inferred_doc_vec = model.infer_vector(text)
    print(f"tags: {tags}, text: {text}")
    print(f"trained_doc_vec: {trained_doc_vec}")
    print(f"inferred_doc_vec: {inferred_doc_vec}")
    print("--"*20)
    # train set 수가 적어서, train과 infer이 조~금 다름 그래두 뭐 비슷함

In [ ]:
doc1 = '비싸지 않아서 좋아요. 오랫동안 촉촉하고 유분감은 느껴지지 않아요. 좋은 성분이 많이 들어있어요. ' 
doc2 = '너무 마음에 들어요. 샘플을 줘서 좋았고 제가 여드름이 좀 나고 열감이 자주 오르는 피부인데 잘 맞았어요. 끈적이지 않고 촉촉해서 답답하지 않아서 좋아요. 흡수도 잘 되고, 바르면 겉은 보송하고 속은 촉촉해요. 복합성이나 수분 부족형에게 추천입니다.'

In [ ]:
docs = [doc1, doc2]

In [ ]:
docs = [doc1, doc2]
new_documents = [
    ['오랫동안', '유분', '성분'], 
    ['마음', '샘플', '제', '여드름', '열감', '피부', '흡수', '복합', '수분', '부족', '추천']
]
# predict training set with its infering vector)
for i, text in enumerate(new_documents): 
    inferred_v = model.infer_vector(text)
    # 현재 doc를 모델을 사용하여 벡터화할때의 값 
    print(docs[i])
    print(f"vector of {text}: {inferred_v}")
    # 기학습된 문서중에서 현재 벡터와 가장 유사한 벡터를 가지는 문서를 topn만큼 추출합니다. 
    most_similar_docs = model.docvecs.most_similar([inferred_v], topn=3)
    # index와 그 유사도를 함께 보여줍니다. 
    # index(tag)가 아닌 문서를 바로 보여주기는 어려운 것 같고, 
    for index, similarity in most_similar_docs:
        print(df.iloc[int(index[4:]),0])
        
        print(f"{index}, similarity: {similarity}")
    #print(most_similar_docs)
    print("=="*20)

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics


In [ ]:
model.docvecs.vectors_docs

In [ ]:
k_list = list(range(2,10))
s_score_list = []

for k in k_list:
  print('k:',k)

  Clustering_Method = KMeans(n_clusters=k, random_state=0)
  X = model.docvecs.vectors_docs
  Clustering_Method.fit(X)

  # 분포 확인 
  print(pd.DataFrame(Clustering_Method.labels_)[0].value_counts())

  # silhouette_score
  s_score = metrics.silhouette_score(X, labels = Clustering_Method.labels_, metric='euclidean', random_state=0)
  s_score_list.append(s_score)
  print('silhouette_score:',s_score)

  print('======================')

  # 분포가 균일하지 않음. 특정 라벨만 너무 많다.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(k_list, s_score_list)
# elbow point가 없다. 

In [ ]:
Clustering_Method = KMeans(n_clusters=5, random_state=0)
X = model.docvecs.vectors_docs
Clustering_Method.fit(X)

# 분포 확인 
print(pd.DataFrame(Clustering_Method.labels_)[0].value_counts())

In [ ]:
cluster_dict = {i:[] for i in range(0, 5)}
for text_tags, label in zip(common_texts_and_tags, Clustering_Method.labels_):
    text, tags = text_tags
    cluster_dict[label].append(text)

cluster_num = []
for label, lst in cluster_dict.items():
    print(f"Cluster {label}")
    for x in lst:
        cluster_num.append(label)
        print(x)

In [ ]:
# 문서의 feature(단어별) cluster_centers_확인해보자
cluster_centers = Clustering_Method.cluster_centers_
print(cluster_centers.shape)
print(cluster_centers)
# shape의 행은 클러스터 레이블, 열은 벡터화 시킨 feature(단어들)

In [ ]:
len(cluster_num)

In [ ]:
cluster_num

In [ ]:
df['cluster_label'] = cluster_num

In [ ]:
df

In [ ]:
Clustering_Method.cluster_centers_.argsort()[:,::-1]

In [ ]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    # 각 클러스터 레이블별 feature들의 center값들 내림차순으로 정렬 후의 인덱스를 반환
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 클러스터 레이블별로 
    for cluster_num in range(cluster_num):
        # 개별 클러스터별 정보를 담을 empty dict할당
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # 각 feature별 center값들 정렬한 인덱스 중 상위 10개만 추출
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        # top_ftr_idx를 활용해서 상위 10개 feature들의 center값들 반환
        # 반환하게 되면 array이기 떄문에 리스트로바꾸기
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        # cluster_details 딕셔너리에다가 개별 군집 정보 넣어주기
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        # 해당 cluster_num으로 분류된 파일명(문서들) 넣어주기
        filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['리뷰']
        # filenames가 df으로 반환되기 떄문에 값들만 출력해서 array->list로 변환
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
    
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print(f"Cluster {cluster_num}으로 분류된 문서들:\n{cluster_detail['filenames'][:5]}")
        print('-'*20)

#feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=Clustering_Method,
                                     cluster_data=df,
                                     feature_names=my_list,
                                     cluster_num=5,
                                     top_n_features=10)
print_cluster_details(cluster_details)

In [ ]:
model.